In [1]:
import cv2
import numpy as np
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Open the video file
video_path = 'test_video.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video was successfully opened
if not cap.isOpened():
    print(f"Error: Cannot open video file {video_path}")
    exit()

# Read the first frame
ret, prev_frame = cap.read()
if not ret or prev_frame is None:
    print("Error: Failed to read the first frame of the video")
    exit()

# Convert the first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Define class names for annotation
class_names = {2: 'Car', 3: 'Motorcycle', 5: 'Bus', 7: 'Truck'}

while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Error: Failed to read a frame from the video")
        break

    # Convert current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Get the vehicle bounding boxes for the current frame
    results = model(frame)
    detections = results.pred[0]
    
    # Filter out relevant classes: car, motorcycle, bus, truck
    vehicle_classes = torch.tensor([2, 3, 5, 7])  # Class indices for car, motorcycle, bus, truck
    vehicle_detections = detections[torch.isin(detections[:, -1], vehicle_classes)]

    # Analyze flow for each detected vehicle
    for detection in vehicle_detections:
        x1, y1, x2, y2 = map(int, detection[:4])  # Convert only coordinates to integers
        conf = float(detection[4])  # Keep confidence as a float
        cls_id = int(detection[5])  # Class ID as integer

        # Extract flow in the bounding box region
        region_flow = flow[y1:y2, x1:x2]

        # Calculate magnitude and angle of flow vectors
        mag, ang = cv2.cartToPolar(region_flow[..., 0], region_flow[..., 1])

        # Optionally, calculate average magnitude and direction
        avg_mag = np.mean(mag)
        avg_ang = np.mean(ang)

        # Draw the bounding box for the detected vehicle
        label = class_names.get(cls_id, 'Unknown')
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw an arrow representing the average flow direction within the bounding box
        arrow_length = int(avg_mag * 10)  # Scale the arrow length for better visibility
        end_x = int(x1 + arrow_length * np.cos(avg_ang))
        end_y = int(y1 + arrow_length * np.sin(avg_ang))
        cv2.arrowedLine(frame, (x1, y1), (end_x, end_y), (0, 255, 0), 2)

    # Update the previous frame
    prev_gray = gray

    # Display the current frame with optical flow annotations
    cv2.imshow('Optical Flow', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master


requirements: Ultralytics requirement ['pillow>=10.3.0'] not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 2.6/2.6 MB 4.6 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0

requirements: AutoUpdate success  8.7s, installed 1 package: ['pillow>=10.3.0']
requirements:  Restart runtime or rerun command for updates to take effect



YOLOv5  2024-9-10 Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument test_elements in method wrapper_CUDA_isin_Tensor_Tensor)

In [3]:
import cv2
import numpy as np
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.cuda()  # Move the model to GPU if available

# Open the video file
video_path = 'test_video.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video was successfully opened
if not cap.isOpened():
    print(f"Error: Cannot open video file {video_path}")
    exit()

# Read the first frame
ret, prev_frame = cap.read()
if not ret or prev_frame is None:
    print("Error: Failed to read the first frame of the video")
    exit()

# Convert the first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Define class names for annotation
class_names = {2: 'Car', 3: 'Motorcycle', 5: 'Bus', 7: 'Truck'}

while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Error: Failed to read a frame from the video")
        break

    # Convert current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Get the vehicle bounding boxes for the current frame
    results = model(frame)
    detections = results.pred[0]

    # Filter out relevant classes: car, motorcycle, bus, truck
    vehicle_classes = torch.tensor([2, 3, 5, 7]).cuda()  # Move vehicle_classes to the same device as detections
    vehicle_detections = detections[torch.isin(detections[:, -1], vehicle_classes)]

    # Analyze flow for each detected vehicle
    for detection in vehicle_detections:
        x1, y1, x2, y2 = map(int, detection[:4])  # Convert only coordinates to integers
        conf = float(detection[4])  # Keep confidence as a float
        cls_id = int(detection[5])  # Class ID as integer

        # Extract flow in the bounding box region
        region_flow = flow[y1:y2, x1:x2]

        # Calculate magnitude and angle of flow vectors
        mag, ang = cv2.cartToPolar(region_flow[..., 0], region_flow[..., 1])

        # Optionally, calculate average magnitude and direction
        avg_mag = np.mean(mag)
        avg_ang = np.mean(ang)

        # Draw the bounding box for the detected vehicle
        label = class_names.get(cls_id, 'Unknown')
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw an arrow representing the average flow direction within the bounding box
        arrow_length = int(avg_mag * 10)  # Scale the arrow length for better visibility
        end_x = int(x1 + arrow_length * np.cos(avg_ang))
        end_y = int(y1 + arrow_length * np.sin(avg_ang))
        cv2.arrowedLine(frame, (x1, y1), (end_x, end_y), (0, 255, 0), 2)

    # Update the previous frame
    prev_gray = gray

    # Display the current frame with optical flow annotations
    cv2.imshow('Optical Flow', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-9-10 Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\hp/.cache\torch\

In [2]:
print(flow)

[[[ 3.6342e-06  1.4552e-05]
  [ 6.7118e-07   2.333e-05]
  [  3.001e-06  2.0699e-05]
  ...
  [  5.014e-05  0.00012527]
  [ 3.4208e-05  0.00012005]
  [ 2.8104e-05  0.00010523]]

 [[ 6.3847e-06  1.1865e-05]
  [ 2.3953e-06  2.3208e-05]
  [ 5.9255e-06  1.8047e-05]
  ...
  [ 5.7715e-05  0.00017997]
  [ 4.7603e-05  0.00017962]
  [ 4.7416e-05   0.0001663]]

 [[  8.321e-06  9.7947e-06]
  [ 4.5389e-06  2.1926e-05]
  [ 7.1577e-06  1.6956e-05]
  ...
  [ 7.5102e-05  0.00022397]
  [ 6.5615e-05  0.00023168]
  [ 6.9644e-05  0.00022541]]

 ...

 [[   -0.29239       0.211]
  [   -0.25269     0.18177]
  [   -0.21205     0.15373]
  ...
  [ -0.0039647   0.0024368]
  [ -0.0061589   0.0013012]
  [  -0.010941 -5.0009e-05]]

 [[   -0.25037     0.18397]
  [   -0.21347     0.15593]
  [   -0.17692     0.12936]
  ...
  [ -0.0050851   0.0014498]
  [ -0.0078874  0.00048927]
  [  -0.013086 -0.00058621]]

 [[  -0.075831     0.05834]
  [  -0.058833    0.044854]
  [  -0.045398    0.034723]
  ...
  [ -0.0053684   0.00104